## HTTP请求
- **请求行  具体的请求类别**
      GET           /         HTTP/1.1
      请求种类   请求内容    协议版本
  - HTTP协议中共定义了八种方法或者叫“动作”来表明对Request-URI指定的资源的不同操作方式，  
    具体介绍如下： 

    - **OPTIONS**：返回服务器针对特定资源所支持的HTTP请求方法。也可以利用向  
      Web服务器发送'\*'的请求来测试服务器的功能性。 
    - **HEAD**：向服务器索要与GET请求相一致的响应，只不过响应体将不会被返回。  
      这一方法可以在不必传输整个响应内容的情况下，就可以获取包含在响应消息头中的元信息。 
    - **GET**：向特定的资源发出请求。 
    - **POST**：向指定资源提交数据进行处理请求（例如提交表单或者上传文件）。  
      数据被包含在请求体中。POST请求可能会导致新的资源的创建和/或已有资源的修改。 
    - **PUT**：向指定资源位置上传其最新内容。 
    - **DELETE**：请求服务器删除Request-URI所标识的资源。 
    - **TRACE**：回显服务器收到的请求，主要用于测试或诊断。 

    - **CONNECT**：HTTP/1.1协议中预留给能够将连接改为管道方式的代理服务器。

  - 虽然HTTP的请求方式有8种，但是我们在实际应用中常用的也就是get和post，其他请求方式也  
    都可以通过这两种方式间接的来实现
     
- **请求头   对请求内容的具体描述**
  - 以键值对的形式对请求信息进行描述
        Accept:text/html
- 空行
- **请求体  具体的请求参数或者提交内容**

## HTTP响应  Response
- **响应行  反馈具体的响应情况**
      HTTP/1.1    200      OK
      版本信息   响应码   附加信息 
      响应码：
      响应码种类:
            1xx    提示信息，表示请求已经接受
            2xx    响应成功 
            3xx    响应需要重定向
            4xx    客户端错误
            5xx    服务端错误
      常见响应码：     
            200  成功
            404  请求内容不存在
            401  没有访问权限
            500  服务器发生未知错误
            503  服务器暂时不能执行
       

- **响应头  对响应的具体描述**
  - Accept-Ranges: bytes
  - 以键值对方式给出响应信息的具体描述

- **空行**
- **响应体   将客户想要的内容进行返回**
      要求：知道什么是HTTP协议，功能特点
 
## 基础的http服务流程程序
1. 接受http请求
2. 给出一定的响应

## IO input  output
- 凡是在内存中存在数据交换的操作都可以认为是IO操作
      比如：
      内存和磁盘交互  读写操作 read  write
      内存和网络交互  recv  send
      内存和终端交互  print  input
- **IO密集型程序**：程序执行中大量的IO操作，而较少的cpu运算。消耗cpu较少，运行时间长

- **CPU密集型程序（计算密集型）**：程序中大量的操作都需要cpu运算，IO操作较少。消耗cpu大，运行速度快

## IO 分类
- 阻塞IO 非阻塞IO IO多路复用  事件IO   异步IO.... 

## 阻塞IO ：
- **默认形态    效率很低的一种IO**

- **阻塞情况**：
  - **因为某种条件没有达到造成的阻塞**  
    e.g. input  accept  recv
  - **处理IO事件的时间消耗较长带来阻塞**  
    e.g.  文件的读写过程，网络数据发送过程

### 非阻塞IO :
- 通过修改IO事件的属性，使其变为非阻塞的状态。（改变了第一种阻塞的状况)

- **非阻塞IO往往和循环搭配使用，这样可以不断执行部分需要**
  
```python
    from socket import *
    from time import sleep,ctime
    s = socket()
    s.bind(('127.0.0.1', 8889))
    s.listen(5)
    # 设置s为非阻塞
    s.setblocking(False)
    while True:
        print('Waiting for connect...')
        try:
            connfd, addr = s.accept()
        except BlockingIOError:
            sleep(2)
            print(ctime())
            continue
        print('Connect from',addr)
        while True:
            data = connfd.recv(1024).decode()
            if not data:
                break
            print(data)
            connfd.sendall(ctime().encode())
        connfd.close()
    s.close()
    ----------------------------------------------------------------------
    Waiting for connect...
    Tue Jul 10 14:50:42 2018
    Waiting for connect...
    Tue Jul 10 14:50:44 2018
    Waiting for connect...
    Tue Jul 10 14:50:46 2018
    Waiting for connect...
```
- **s.setblocking()**
  - 功能： 将套接字设置为非阻塞状态
  - 参数： bool   设置为False则表示设置为非阻塞

### 超时检测
  - 将原本阻塞的IO设置一个最长阻塞等待时间，在规定时间如果达到条件则正常执行，如果  
    时间到仍未达到条件则结束阻塞。
```python
    from socket import *
    from time import sleep,ctime
    s = socket()
    s.bind(('127.0.0.1', 8889))
    s.listen(5)
    # 设置超时等待时间
        s.settimeout(5)

    while True:
        print('Waiting for connect...')
        try:
            connfd, addr = s.accept()
        except timeout:
            print("time out...")
            sleep(2)
            print(ctime())
            continue
        print('Connect from',addr)
        while True:
            data = connfd.recv(1024).decode()
            if not data:
                break
            print(data)
            connfd.sendall(ctime().encode())
        connfd.close()
    s.close()
    -------------------------------------------------
    Waiting for connect...
    time out...
    Tue Jul 10 15:02:04 2018
    Waiting for connect...
    time out...
    Tue Jul 10 15:02:11 2018
    Waiting for connect...
    time out...
```
- **s.settimeout(sec)**
  - 功能 ： 设置套接字超时时间
  - 参数 ： 设置的时间

### IO多路复用  select
- 定义： 同时监控多个IO事件，当哪个IO事件准备就绪就执行哪个IO事件。此时形成多个IO时间  
  都可以操作的现象，不必逐个等待执行的效果。
- 准备就绪：IO事件即将发生的临界状态

```python
    import select

    select   # ----》 windows  linux  unix
    poll    # ---》 linux unix
    epoll    # --》 linux  unix
    help(select.select)
----------------------------------------------------------------------
     select(rlist, wlist, xlist[, timeout]) -> (rlist, wlist, xlist)
   
```
- r, w, x = select(rlist, wlist, xlist[, timeout]) 
  - 功能：监控IO事件，阻塞等待IO事件的发生
  - 参数: 
        rlist  列表  存放我们监控等待处理的IO事件
        wlist  列表  存放我们要主动处理的IO事件
        xlist  列表  存放如果发生异常需要我们处理的
        timeout 数字  超时时间
  - 返回值：
        r  列表   rlist当中准备就绪的IO
        w  列表   wlist当中准备就绪的IO
        x  列表   xlist当中准备就绪的IO
        
```python

    from socket import *
    from  select import select
    import sys
    #创建tcp套接字作为关注的IO
    s = socket()
    s.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
    s.bind(('127.0.0.1',8889))
    s.listen(5)


    # 三个关注列表
    rlist = [s]
    wlist = []
    xlist = [s]
    while True:
        # 提交关注的IO事件，等待处理
        print('waiting for IO...')
        rs,ws,xs = select(rlist,wlist,xlist,3)
        for r in rs:
            if r is s:
                connfd,addr = r.accept()
                print('connect from',addr)
                # 增加关注事件
                rlist.append(connfd)
            else:
                data = r.recv(1024).decode()
                if not data:
                    rlist.remove(r)
                    r.close()
                print('Receive:',data)
                # 加入到要处理的IO事件列表
                wlist.append(r)
                # r.send(b'Receive your message')
        for w in ws:
            w.send('这是一条回复消息'.encode())
            wlist.remove(w)

        # 处理发生异常
        for x in xs:
            if x is s:
                s.close()
                sys.exit(0)
```
- **注意事项**：
  - IO多路复用处理IO的过程中不应有死循环出现，使一个客户端长期占有服务端
  - IO多路复用是一种并发行为，但是是单进程程序，效率较高

## 位运算
- 按照二进制位进行操作运算
        & (按位与)    一0则0
        |（按位或）   一1则1
        ^（按位异或）  相同为0不同为1
        <<（左移）    右侧补0
        >>（右移）    挤掉低位的数字
- 使用 ： 
  1. 在做底层硬件的寄存器操作
  2. 在做标志位过滤时


# 作业：
- 写一个select服务端，同时关注客户端连接，客户端发送消息，和终端输入。将客户端发送内容  
  和终端输入的内容都写按行到一个文件里




In [ ]:
from socket import *
from select import  select
import  sys
s = socket()
s.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
s.bind(('127.0.0.1',8889))
s.listen(5)

rlist = [s,sys.stdin]
wlist = []
xlist = [s]

f = open('test.txt','w')

while True:
    rs,ws,xs = select(rlist,wlist,xlist)
    for r in rs:
        if r is s:
            c,addr = r.accept()
            rlist.append(c)
        elif r is sys.stdin:
            data = r.readline()
            f.write(data)
        else:
            data = r.recv(1024)
            if not data:
                rlist.remove(r)
            else:
                f.write(data.decode()+'\n')